<a href="https://colab.research.google.com/github/RajRover/ML_Projects/blob/main/Wildfire_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data_set3=pd.read_csv('/content/modis_2024_India.csv')

In [ ]:

data_set3=data_set3[['latitude','longitude','acq_date']]

In [ ]:
print(data_set3)

       latitude  longitude    acq_date
0       23.7690    86.3923  2024-01-01
1       34.0996    74.9958  2024-01-01
2       31.0805    78.2029  2024-01-01
3       34.0908    74.9942  2024-01-01
4       23.6291    74.3663  2024-01-01
...         ...        ...         ...
74024   22.0444    83.7350  2024-12-31
74025   23.5604    87.2335  2024-12-31
74026   23.6847    86.3956  2024-12-31
74027   23.6663    86.9220  2024-12-31
74028   23.7578    86.4017  2024-12-31

[74029 rows x 3 columns]


In [ ]:
data_set3.tail()

,latitude,longitude,acq_date
74024,22.0444,83.7350,2024-12-31
74025,23.5604,87.2335,2024-12-31
74026,23.6847,86.3956,2024-12-31
74027,23.6663,86.9220,2024-12-31
74028,23.7578,86.4017,2024-12-31


In [ ]:
data_set3.head()

,latitude,longitude,acq_date
0,23.7690,86.3923,2024-01-01
1,34.0996,74.9958,2024-01-01
2,31.0805,78.2029,2024-01-01
3,34.0908,74.9942,2024-01-01
4,23.6291,74.3663,2024-01-01


In [ ]:
data_set3['acq_date'].value_counts()

,count
acq_date,
2024-03-14,1215
2024-04-06,1033
2024-03-16,985
2024-03-11,961
2024-11-18,886
...,...
2024-08-25,2
2024-08-08,1
2024-08-07,1


In [ ]:
data_set3.shape

(74029, 3)

In [ ]:
data_set3['acq_date'] = pd.to_datetime(data_set3['acq_date'], format='%Y-%m-%d')

In [ ]:
data_set = data_set3[data_set3['acq_date'] > '2024-10-20']

In [ ]:
data_set.head()

,latitude,longitude,acq_date
55835,28.9280,75.8648,2024-10-21
55836,31.4089,76.2722,2024-10-21
55837,31.4069,76.2840,2024-10-21
55838,29.1735,75.7660,2024-10-21
55839,28.9260,75.8766,2024-10-21


In [ ]:
print(data_set['acq_date'])

55835   2024-10-21
55836   2024-10-21
55837   2024-10-21
55838   2024-10-21
55839   2024-10-21
           ...    
74024   2024-12-31
74025   2024-12-31
74026   2024-12-31
74027   2024-12-31
74028   2024-12-31
Name: acq_date, Length: 18194, dtype: datetime64[ns]


In [ ]:
import requests
from tqdm import tqdm
import time

In [ ]:
API_KEY='a2b644e624204bedb01144935251110'

In [ ]:
def get_weather_history(lat, lon, date):


    url = f"https://api.weatherapi.com/v1/history.json?key={API_KEY}&q={lat},{lon}&dt={date}"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        day_data = data["forecast"]["forecastday"][0]["day"]

        avgtemp_c = day_data.get("avgtemp_c")
        total_precip_mm = day_data.get("totalprecip_mm")
        avg_humidity = day_data.get("avghumidity")
        wind_kph = day_data.get("maxwind_kph")

        # Compute average pressure from hourly data
        hourly_data = data["forecast"]["forecastday"][0]["hour"]
        pressures = [h["pressure_in"] for h in hourly_data if "pressure_in" in h]
        pressure_in = sum(pressures) / len(pressures) if pressures else None

        return avgtemp_c, total_precip_mm, avg_humidity, pressure_in, wind_kph

    except Exception as e:
        print(f"⚠️ Error for {lat},{lon} on {date}: {e}")
        return None, None, None, None, None


In [ ]:
from tqdm import tqdm
import time

API_KEY = 'a2b644e624204bedb01144935251110'

results = []

for _, row in tqdm(data_set.iterrows(), total=len(data_set)):
    lat = row['latitude']
    lon = row['longitude']
    date = (str)(row['acq_date']).split()[0]
    weather_data = get_weather_history(lat, lon, date)

    results.append(weather_data)
    time.sleep(1)


  3%|▎         | 519/18194 [09:20<5:32:59,  1.13s/it]

⚠️ Error for 26.695,79.3406 on 2024-10-26: HTTPSConnectionPool(host='api.weatherapi.com', port=443): Read timed out. (read timeout=10)


 70%|███████   | 12820/18194 [3:51:48<1:37:40,  1.09s/it]

⚠️ Error for 25.105,83.8855 on 2024-12-07: 502 Server Error: Bad Gateway for url: https://api.weatherapi.com/v1/history.json?key=a2b644e624204bedb01144935251110&q=25.105,83.8855&dt=2024-12-07


 89%|████████▊ | 16144/18194 [4:51:49<40:18,  1.18s/it]

⚠️ Error for 21.549,82.2463 on 2024-12-19: 502 Server Error: Bad Gateway for url: https://api.weatherapi.com/v1/history.json?key=a2b644e624204bedb01144935251110&q=21.549,82.2463&dt=2024-12-19


100%|██████████| 18194/18194 [5:28:55<00:00,  1.08s/it]


In [ ]:
results

[(27.8, 0.0, 31, 29.841250000000002, 12.2),
 (21.8, 0.0, 46, 29.908749999999998, 10.4),
 (21.8, 0.0, 46, 29.908749999999998, 10.4),
 (27.8, 0.0, 30, 29.840416666666666, 11.5),
 (27.8, 0.0, 31, 29.841250000000002, 12.2),
 (27.8, 0.0, 31, 29.841250000000002, 12.2),
 (27.8, 0.0, 31, 29.841250000000002, 12.2),
 (31.2, 0.0, 17, 29.817916666666665, 13.7),
 (31.2, 0.0, 17, 29.817916666666665, 13.7),
 (26.5, 0.0, 52, 29.881249999999998, 13.0),
 (26.5, 0.0, 52, 29.881249999999998, 13.0),
 (28.3, 10.9, 60, 29.835416666666664, 10.4),
 (26.5, 0.0, 52, 29.881249999999998, 13.0),
 (24.7, 8.4, 80, 29.886666666666667, 8.3),
 (26.0, 13.93, 86, 29.847916666666666, 7.2),
 (23.0, 0.0, 75, 29.869583333333335, 5.0),
 (28.4, 3.5, 74, 29.81625, 14.4),
 (27.8, 3.8, 76, 29.8175, 12.6),
 (26.1, 19.8, 88, 29.844166666666666, 7.9),
 (26.3, 14.27, 81, 29.83375, 8.6),
 (26.1, 19.8, 88, 29.844166666666666, 7.9),
 (26.1, 19.8, 88, 29.844166666666666, 7.9),
 (25.6, 4.54, 88, 29.860833333333332, 10.8),
 (27.4, 0.1, 68, 

In [ ]:
columns = ["avgtemp_c", "total_precip_mm", "avg_humidity", "pressure_in", "wind_kph"]

weather_df = pd.DataFrame(results, columns=columns)


In [ ]:
weather_df.head()

,avgtemp_c,total_precip_mm,avg_humidity,pressure_in,wind_kph
0,27.8,0.0,31.0,29.841250,12.2
1,21.8,0.0,46.0,29.908750,10.4
2,21.8,0.0,46.0,29.908750,10.4
3,27.8,0.0,30.0,29.840417,11.5
4,27.8,0.0,31.0,29.841250,12.2


In [ ]:
weather_df = weather_df.reset_index(drop=True)
df = pd.concat([data_set, weather_df], axis=1)

In [ ]:
df = df.dropna(axis=1, how='all')

In [ ]:
df.head()

,latitude,longitude,acq_date,avgtemp_c,total_precip_mm,avg_humidity,pressure_in,wind_kph
0,28.9280,75.8648,2024-10-21,27.8,0.0,31.0,29.841250,12.2
1,31.4089,76.2722,2024-10-21,21.8,0.0,46.0,29.908750,10.4
2,31.4069,76.2840,2024-10-21,21.8,0.0,46.0,29.908750,10.4
3,29.1735,75.7660,2024-10-21,27.8,0.0,30.0,29.840417,11.5
4,28.9260,75.8766,2024-10-21,27.8,0.0,31.0,29.841250,12.2


In [ ]:
df.to_csv("filename.csv", index=False)

In [ ]:
from google.colab import files

In [ ]:
files.download('filename.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>